## Problem Set 1

_csc427, semester 212
<br>
university of miami
<br>
date: 31 january 2021
<br>
updated: 3 february 2021_


---

### Student Name:  _____

---

### The Python you need to know

We will introduce two important Python datastructures: the _tuple_ and the _dictionary_. 

The _tuple_ is an sequence of elements, indexed by integers starting at 0. In this way is is like a list, except that a tuple it is _immutable_ &mdash; once defined it cannot be changed. It is created by enclosing a comma separated sequence of elements in parenthesis. For a one element tuple, a comma will follow the single element, to distinguish the notation from just a element gratuitously enclosed in parenthesis. Elements from the tuple are accessed using the square bracket notation, but and element cannot be assigned into.

The _dictionary_ is a collection of _key:value_ pairs, where the key is any immutable type. (Hence our interest in tuples, because they can be used as keys, but lists cannot.) The dictionary is mutable, and at it can be used as a sequence, in which case its _keys_ are sequentially provided in some order (I believe in insertion order). Dictionaries use curly braces around a comma seperated sequence of key:value pairs, with a colon (:) separating the key and value. Access or update to the value associated to a key uses the sequare bracket notation.


We represent the transition function as a dictionary where $\delta(S,c)=T$ is represented as the _key:value_ pair  (S,c):T, where S, c and T are strings, and (S,c) is a tuple.

In [1]:
# a tuple, an immutable sequence. 

states = ['A','B','C']
alphabet = ['a','b','c']

transition_key = ('A','c')  # meaning in state A, with letter c

# a dictionary, a mapping. the key must be immutable.

transitions = {('A','a'):'A', ('A','b'):'B',  # from A, go to B is b is seen
               ('B','a'):'B', ('B','b'):'B'   # from B, stay in B
              }

print("Some of the types:")
print('\t* The type of a state is:', type('A'))
print('\t* The type of a collection of states is:', type(states))
print('\t* The type of a state-letter pair is:', type(transition_key))
print('\t* The type of a transition object is:',type(transitions))

print("Here are some transitions:")
print('\t* For state','A','and character','b','the resulting state is',transitions[('A','b')])

Some of the types:
	* The type of a state is: <class 'str'>
	* The type of a collection of states is: <class 'list'>
	* The type of a state-letter pair is: <class 'tuple'>
	* The type of a transition object is: <class 'dict'>
Here are some transitions:
	* For state A and character b the resulting state is B


### The Machine Model

Problem Set 0 has code that modeled a Finite State Automata, a DFA.

That realization was not very structured about how DFA's are represented, especially how the transition function is implemented. We will now give a machine description that follows the definition of a DFA as given in our textbook by Sipser. There a DFA is the 5-tuple,

$$
( Q, \Sigma, \delta, q_0, F)
$$

where $Q$ is the set of states, $\Sigma$ the set of letters, $\delta$ is the transition function,

$$
\delta: Q \times \Sigma \longrightarrow Q
$$

the start state is $q_0\in Q$ and $F\subseteq Q$ is the set of accepting states.

This is represented in Python with states and charaters as strings; $Q$, $\Sigma$ and $F$ as lists of strings;
and $\delta$ as a dictionary with keys being a 2-tuples of strings.


I also define a TestMachine class, that encapsulates the test functions to test a machine.

__Note:__ The verbose global can be made true or false, and if true some debugging messages will be printed. Remember to re-run the cell on changing the vale of verbose.


In [2]:
verbose = False

class MachineModel:
    """
    A machine description is a dictionary with,
        states: a list of states
        alphabet: a list of letters
        transitions: a dictionary with keys tuples (a state,a letter) to a state
        start: a state (the start state)
        accept: a list of states (the accepting states)
        
    """
    
    def __init__(self,machine_description):
        self.states = machine_description['states']
        self.alphabet = machine_description['alphabet']
        self.transitions = machine_description['transitions']
        self.start_state = machine_description['start'] 
        self.accept_states = machine_description['accept']
        self.current_state = self.start_state 

    def do_transition(self,letter):
        self.current_state = self.transitions[(self.current_state,letter)]
    
    def compute(self,word):
        self.current_state = self.start_state
        if verbose : print(self.current_state)
        for w in word:
            self.do_transition(w)
            if verbose : print(w,self.current_state)
        return self.current_state in self.accept_states


class TestMachine:
    """
    A class that constructs a MachineModel from a Machine Description, 
    and can run a test. The test object is a list of pairs, 
        [(string,case), ... ]
    where case is True or False, if the string is, or is not, in the 
    language.
    """
    
    def __init__(self,machine_description):
        self.machine = MachineModel(machine_description)
      
    def run(self,tests):
        print('running tests ...')
        for (t,r) in (tests):
            if self.machine.compute(t) != r:
                print(r,'\t|'+t+'|','\tWRONG, ABORT')
                return False
            print(r,'\t|'+t+'|','\tOK')
        return True


### Example machines




_**Define a machine such that:**_ 
A machine over the alphabet 'a' and 'b' that accepts exactly when the string is without any 'b', including
the empty string.

_**Strategy:**_

1. To design a DFA to solve a language problem, one first focuses on finding and carefully defining what states will be needed. Here it seems that two states will do: one that remembers that an 'b' has not yet been found in the string, and another for otherwise. This seems to be both sufficient (enough to get the job done) and necessary (nothing less will work).

2. Having identified the states, we then consider the start state and the transitions, and then lastly we consider the accept states. A single transition function can yield numerous related languages. For instance, just by changing which are the accept states, we can _complement_ a language.

3. Naming the states 'A' and 'B', hopefully mnenomically only 'a's being seen and a 'b' being seen, we begin to build the Machine Description.



In [3]:

# this one remembers if a b was ever seen

machine_never_b = { 
    'states':['A','B'], 
    'alphabet':['a','b'], 
    'transitions':{('A','a'):'A', ('A','b'):'B', ('B','a'):'B', ('B','b'):'B'},
    'start':'A', 
    'accept':['A']
}

# some sample strings in and not in the language

tests = [('',True), # the empty string is accepted
        ('a',True), ('b',False), # single letters
        ('aaaa',True), ('aaba',False)
        ]

tm = TestMachine(machine_never_b)
tm.run(tests)


running tests ...
True 	|| 	OK
True 	|a| 	OK
False 	|b| 	OK
True 	|aaaa| 	OK
False 	|aaba| 	OK


True


_**Define a machine such that:**_ 
A machine over the alphabet 'x' and 'y' such that the string is length at least three, and the third to last character is an 'x'.

_**Strategy:**_

1. What states are _necessary_ and _sufficient_ to solve the problem.
2. Name those states (best do this mnemonically.
3. Load up the transition function and decide the start state.
4. Announce the accept states.


We have to know whether the string is at least 3 in length, and it follows we need to know when number of letters so far is 0, 1, 2, or 3 or more. This way (a) we know, Yes or No, and (b) we can keep track of going from No to Yes.

We have to know the third to last letter, at least 'x' or not x, but this is the same as 'x' or 'y'. But to
get to that we need to also know the second to last and last letter, because these could turn into the third to last.

Mnemonics for states are strings in X and Y, of length 1 through 3, represented the up to last three characters
read; as well as state E for the empty string.



In [4]:
## third from last letter an x

machine_never_b = { 
    'states':['E','X','Y','XX','XY','YX','YY',
             'XXX','XXY','XYX','XYY','YXX','YXY','YYX','YYY'], 
    'alphabet':['x','y'], 
    'transitions':{
            ('E','x'):'X', ('E','y'):'Y',
            ('X','x'):'XX', ('X','y'):'XY',
            ('Y','x'):'YX', ('Y','y'):'YY',
            ('XX','x'):'XXX', ('XX','y'):'XXY',
            ('XY','x'):'XYX', ('XY','y'):'XYY',
            ('YX','x'):'YXX', ('YX','y'):'YXY',
            ('YY','x'):'YYX', ('YY','y'):'YYY',
            ('XXX','x'):'XXX', ('XXX','y'):'XXY',
            ('XXY','x'):'XYX', ('XXY','y'):'XYY',
            ('XYX','x'):'YXX', ('XYX','y'):'YXY',
            ('XYY','x'):'YYX', ('XYY','y'):'YYY',
            ('YXX','x'):'XXX', ('YXX','y'):'XXY',
            ('YXY','x'):'XYX', ('YXY','y'):'XYY',
            ('YYX','x'):'YXX', ('YYX','y'):'YXY',
            ('YYY','x'):'YYX', ('YYY','y'):'YYY',         
        },
    'start':'E', 
    'accept':['XXX','XXY','XYX','XYY']
}

# some sample strings in and not in the language

tests = [('',False), # the empty string is accepted
        ('xx',False), ('yy',False), 
        ('xxx',True), ('yxx',False),
         ('yyxyy',True), ('yyxyx',True),
         ('yyyxx',False), ('xxyxx', False)
        ]

tm = TestMachine(machine_never_b)
tm.run(tests)


running tests ...
False 	|| 	OK
False 	|xx| 	OK
False 	|yy| 	OK
True 	|xxx| 	OK
False 	|yxx| 	OK
True 	|yyxyy| 	OK
True 	|yyxyx| 	OK
False 	|yyyxx| 	OK
False 	|xxyxx| 	OK


True

## Exercise A:  

In these exercises, the alphabet is $\{ a, b\}$, and are take from the exercises to chapter 1 of Siper's book,

1. $\{\,w\,|\, w \mbox{ contains at least three a's } \}$
1. $\{\,w\,|\, w \mbox{ has an odd number of a's and ends with a b } \}$
1. $\{\,w\,|\, w \mbox{ contains neither the substrings } ab \mbox{ or } ba\}$
1. $\{\,w\,|\, w \mbox{ is any string not in } a^*b^* \}$
1. $\{\,w\,|\, w \mbox{ is any string in } (ab^*)^* \}$
1. $\{\,w\,|\, w \mbox{ is any string not in } (ab^+)^*a \}$, where $b^+ = bb^*$.

In these exercises, the alphabet is $\{ 0, 1\}$, and are take from the exercises to chapter 1 of Siper's book,

7. $\{\,w\,|\, w \mbox{ begins with a 0 and ends with a 1 } \}$
1. $\{\,w\,|\, w \mbox{ is one of the four strings: 100, 010, 01001, or the empty string } \}$
1. $\{\,w\,|\, w \mbox{ is any string except 00 and 000 } \}$
1. $\{\,w\,|\, w \mbox{ contains the substring  } 110 \}$

In these exercise, the alphabet is $\{ x, y, z\}$. These will need many states, because they have to consider
alternatives. 

11. $\{\,w\,|\, w \mbox{ contains at least two y's and at least two z's} \}$
1. $\{\,w\,|\, w \mbox{ can be written as } YZ \mbox{ where $Y$ has exactly two y's, and $Z$ has exactly two z's} \}$.

__Hint:__ For exercise 12, $yyzyzz$ is in this language, but $yyyzzz$ is not. 
- We can write $yyzyzz$ as $yyz$ followed by $yzz$.
- No matter where we break $yyyzzz$ into two strings, we cannot avoid a violation. 
For instance, $yy$ followed by $yzzz$ will have too may z's in the second string.

__Hint:__ If "not in the language" is confusing, build the machine for "in the language", and then change just the accept states to accept the complement. 

__Hint:__ In situations where a string must meet both condition A and condition B, consider buidling a machine MA for condition A and a machine MB for condition B, and then creating the product machine MAB, defined as follows:

<pre>
THE STATES:
    If MA has states,
        A1, A2, ...  
    and MB has states,
        B1, B2, ...
    then MAB has states,
        A1B1, A1B2, B2A1, B2A2,   ...
    
THE TRANSITIONS:
    For every transition in MA and MB
        (Ax,&sigma;):Ay, (Bu,&sigma;):Bv
    then place in MAB the transition,
        (AxBu,&sigma;):AyBv
        
START STATE:
    If A is the start state for MA, and B is the start state for MA, 
    then AB is the start state for MAB.
</pre>


In [5]:
# the following templates are provided.

# the sample values explain the type by providing empty strings
# to indicate type string, a one element list to indicate type list,
# a one entry dictionary to indicate type dictionary, and a
# sample pair ('','') to indicate a type tuple of two strings.

dfa_md = [None for i in range(13)]

dfa_md[1] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[2] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[3] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[4] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[5] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[6] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[7] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[8] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[9] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[10] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[11] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}

dfa_md[12] = {
    'states': [''],
    'alphabet': [''],
    'transitions': {('',''):''},
    'start': '',
    'accept': ['']
}


In [6]:
# testing
# these are basic tests only. 

dfa_test = [None for i in range(13)]

dfa_test[1] = [
    ('aaba',True),
    ('aabb',False)    
]

dfa_test[2] = [   # notes, interspers b's, test two cases ...
    ('aaab',True),
    ('baa',False)
]

dfa_test[3] = [
    ('aaa',True),
    ('aab',False)
]

dfa_test[4] = [  # b and a are False, and maybe further mix of a's and b's
    ('ba',True),
    ('aa',False)
]

dfa_test[5] = [  # note: must beging with an a or be empty
    ('baaa',False),
    ('abab',True)
]

dfa_test[6] = [  
    ('abba',False),
    ('abab',True)
]

dfa_test[7] = [
    ('01',True),
    ('10',False)
]

dfa_test[8] = [
    ('',True),
    ('11',False)
]
dfa_test[9] = [
    ('000',False),
    ('011',True)
]

dfa_test[10] = [
    ('001110',True),
    ('001010',False)
]

dfa_test[11] = [
    ('zyxyzx',True),
    ('xzxyxx',False)
]

dfa_test[12] = [
    ('yyyzzz',False),
    ('yyzyzz',True)
]



def basic_test():
    
    correct = 0
    num_tests = len(dfa_test)
    for i in range(1,num_tests):
        print("\nExercise",i)
        try:
            tm = TestMachine(dfa_md[i])
            if tm.run(dfa_test[i]):
                correct += 1
        except Exception as exception:
            print("\n*** exception thrown:", str(type(exception)))
    print("correct:",correct,"out of",num_tests-1)

basic_test()


Exercise 1
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 2
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 3
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 4
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 5
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 6
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 7
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 8
running tests ...
True 	|| 	OK

*** exception thrown: <class 'KeyError'>

Exercise 9
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 10
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 11
running tests ...

*** exception thrown: <class 'KeyError'>

Exercise 12
running tests ...

*** exception thrown: <class 'KeyError'>
correct: 0 out of 12
